We start with the source reconstruction problem with given boundary data (ICP) and smooth pressure assumption. Minimization problem:

$$
\min_{g} J(g) = \frac{\gamma_1}{2} \int_0^T \int_{\Gamma_V}(p(x,t) - ICP(x,t))^2 ds \, dt \\
+ \frac{\gamma_2}{2} \int_0^T \int_{\Gamma_{SAS}}(p(x,t) - ICP(x,t))^2 ds \, dt \\
+ \frac{\gamma_3}{2} \int_0^T \int_{\Omega} (\Delta p(x,t))^2 ds \, dt
$$

subject to the Darcy equation with Neumann boundary conditions and inital data:


$$
\begin{aligned}
c \frac{\partial p}{\partial t} + K \Delta p &= g \quad \text{ in } \Omega \times (0,T) \\
K \nabla p \cdot n &= 0 \quad  \text{ on } (\Gamma_V \cup \Gamma_{SAS}) \times (0,T)  \\
p &= 0 \quad \text { in } \Omega \times \{0 \}
\end{aligned}
$$

Since we can not directly compute the laplacian of the first order Lagrange solution of the pressure, we replace $\Delta p$  it with:

$$
\sum_E \int_E [ \nabla p] ds
$$

where $[\cdot]$ denotes the jump term on internal boundaries $E$. Now, we can solve the optimization problem by dolfin-adjoint and some gradient based optimization algorithm (e.g. L-BFGS, Newton-CG, ..)

***Problem: Very slow convergence of iterative gradient descent methods!***
(laplacian regularization term slowly penetrates the domain, only very few points have a nonzero gradient...)

***Alternative: Compute optimal pressure:***

Assume that there extists an optimal pressure $p_{opt}$, such that:

$$
\begin{aligned}
K \Delta p_{opt} &= 0 \quad \quad &&\text{ in } \Omega \times (0,T) \\
 p_{opt} &= ICP \quad &&\text{ on } (\Gamma_V \cup \Gamma_{SAS}) \times (0,T)
\end{aligned}
$$

Then p minimizes the functional and the corresponding source $g_{opt}$ can easily be computed:

$$
g_{opt} = c \frac{\partial p_{opt}}{\partial t} + \underbrace{K \Delta p}_{= 0} =  c \frac{\partial p_{opt}}{\partial t}
$$

***Remarks:***
* source is spatially as smooth as the pressure
* temporal smoothness depends on the boundary conditions
* computationally very efficient: only one laplace solve per timestep and source computation by differencing the obtained pressure

***Open Questions:***
* what about the Neumann BC of the original Darcy constraint? It is not extactly fulfilled by p, but the deviation is small?
* Is the laplacian a good smoothness requirement? (Seems so...)
* generalization to Biot?

In [ ]:
from braininversion.meshes import generate_doughnut_mesh
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from braininversion.DarcySolver import solve_darcy
from braininversion.Optimization import optimize_darcy_source, compute_minimization_target, update_expression_time
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)
from braininversion.reconstructByLaplaceSolve import(solve_laplace_for_all_timesteps,
                                     compute_sources_from_pressures)

In [ ]:
# time stepping
T = 1.2           # final time
num_steps = 10    # number of time steps
dt = T/ num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17 # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-4          # storage coefficient
mmHg2Pa = 132.32
material_parameter = {"K":K, "c":c}

brain_radius = 0.1
ventricle_radius = brain_radius/4
N = 10
mesh, boundary_marker = generate_doughnut_mesh(brain_radius, ventricle_radius, N)

f = 1
A = 2*mmHg2Pa
p_skull = Expression("A*sin(2*pi*f*t)", A=A,f=f,t=0,degree=2)
#p_ventricle = Expression("A*sin(2*pi*f*t)", A=A*0.8,f=f,t=0,degree=2)
p_ventricle = Constant(0.0)

x_coords = np.linspace(ventricle_radius, brain_radius, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

boundary_conditions = {1:{"Neumann":Constant(0.0)}, 2:{"Neumann":Constant(0.0)}}

gamma3 = 1e-5
def laplace(p):
    mesh = p.function_space().mesh()
    n = FacetNormal(mesh)
    return gamma3*jump(grad(p), n)**2

time_dep_expr = [p_skull, p_ventricle]


minimization_target = {"ds": { 1: lambda x: (x - p_ventricle)**2,
                              2: lambda x: (x - p_skull)**2
                             },
                       "dS":{"everywhere":laplace}
                      }

In [ ]:
# compute constant controls as initial guess
res = optimize_darcy_source(mesh, material_parameter, times, minimization_target,
                            boundary_marker, boundary_conditions,
                            time_dep_expr=time_dep_expr, opt_solver="scipy",
                            control_args="constant"
                            )

opt_ctrls, opt_solution, initial_solution = res

# compute controls
res = optimize_darcy_source(mesh, material_parameter, times, minimization_target,
                            boundary_marker, boundary_conditions,
                            time_dep_expr=time_dep_expr,
                            opt_solver="moola_bfgs",
                            optimization_parameters={"jtol":1e-2,
                                                     "maxiter":500,
                                                     "mem_lim":10},
                            #control_args=["DG", 0],
                            initial_guess=opt_ctrls
                           )

opt_ctrls, opt_solution, initial_solution = res


In [ ]:
laplace_bc = {1:{"Dirichlet":p_ventricle}, 2:{"Dirichlet":p_skull}}
         
laplace_pressures = solve_laplace_for_all_timesteps(mesh, boundary_marker, laplace_bc,
                                                   times, time_dep_expr)

laplace_sources = compute_sources_from_pressures(laplace_pressures, c, dt)

laplace_solution = solve_darcy(mesh, laplace_sources, T, num_steps, K,
                              boundary_marker, boundary_conditions,
                              c=c)
laplace_solution = [s.copy() for s in laplace_solution]

J_laplace = 0.0

minimization_target = {"ds": { 1: lambda x: (x - p_ventricle)**2,
                              2: lambda x: (x - p_skull)**2
                             },
                       "dS":{"everywhere":laplace}
                      }
for i, p in enumerate(laplace_solution):
    update_expression_time(time_dep_expr, times[i])
    J_laplace += compute_minimization_target(p, minimization_target, boundary_marker)
print(J_laplace)

In [ ]:
pressures = {"p_opt_const" : extract_cross_section(initial_solution, slice_points)/mmHg2Pa,
             "p_opt" : extract_cross_section(opt_solution, slice_points)/mmHg2Pa,
             "p_laplace":extract_cross_section(laplace_pressures, slice_points)/mmHg2Pa,}

forces = {"f_opt": extract_cross_section(opt_ctrls, slice_points),
          "f_laplace": extract_cross_section(laplace_sources, slice_points)}

style_dict["p_laplace"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["f_laplace"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["f_opt"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["p_opt"] = {"ls":"-.", "lw":3, "color":"green"}


In [ ]:
for i in [2,4,6,8]: 
    plot_pressures_and_forces_cross_section(pressures, forces, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [20, 40 ,60, 80]:
    plot_pressures_and_forces_timeslice(pressures, forces, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")